<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/01_nfl_predict_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 01. EDA (Exploratory data analysis)

In [2]:
PATH = 'https://raw.githubusercontent.com/rafabandoni/nfl-predict/refs/heads/main/data/output/'

In [4]:
games_score_df = pd.read_parquet(PATH + 'games_stats_nfl.parquet')
games_score_df.head()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,...,special_teams_punting_td_away_,special_teams_punting_int_away_,special_teams_punting_rate_away_,special_teams_punting_1st_away_,special_teams_punting_1st_perc_away_,special_teams_punting_20+_away_,special_teams_punting_40+_away_,special_teams_punting_lng_away_,special_teams_punting_sck_away_,special_teams_punting_scky_away_
0,2011-09-08,2011,False,Packers,42,34,Saints,False,True,213,...,33,22,90.8,236,0.357,47,10,-80,26,195
1,2011-09-11,2011,False,Cardinals,28,21,Panthers,False,True,233,...,9,21,57.0,125,0.258,30,3,88,50,346
2,2011-09-11,2011,False,Ravens,35,7,Steelers,False,True,229,...,22,9,95.2,175,0.365,62,11,56,43,289
3,2011-09-11,2011,False,Bears,30,12,Falcons,False,True,216,...,28,9,90.8,200,0.347,32,6,46,23,158
4,2011-09-11,2011,False,Browns,17,27,Bengals,False,False,224,...,26,20,82.4,206,0.349,45,9,-78,28,221
